## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [1]:
import mne
import numpy as np

# Definição das frequências de estimulação

freqs = np.array([6, 6.5, 7, 7.5, 8.2, 9.3])

# Definição do limiar

threshold = 0.2
psds = []
# Isolamento das frequências

# Obtenção do objeto mne.EpochArray

epochs = mne.read_epochs("../../datasets/avi/single/sub1_run1-epo.fif")

# Isolamento das frequências

for freq in freqs:
    # Obtenção do valor mínimo e máximo
    min_freq = freq - threshold
    max_freq = freq + threshold

    # Filtragem dos dados
    epochs_filtered = epochs.filter_data(low_cut=min_freq, high_cut=max_freq)

    # Cálculo do PSD
    psd = epochs_filtered.psd(tmin=0, tmax=0.5, fmin=min_freq, fmax=max_freq)

    # Armazenamento dos resultados
    psds[freq] = psd

# Cálculo das características

features = []
for freq in freqs:
    # Características manuais
    features.append(psds[freq].max())
    features.append(psds[freq].mean())
    features.append(psds[freq].median())

# Classificação

clf = LogisticRegression(solver='lbfgs', max_iter=1000)
clf.fit(features, y)

# Avaliação

score = clf.score(features, y)

print('Score do modelo: %.2f' % score)

FileNotFoundError: File does not exist: "/home/moraski/faculdade/reconhecimento/patter-recog/SSVEP/avi/classifier/../../datasets/avi/single/sub1_run1-epo.fif"